<a href="https://colab.research.google.com/github/sanzidarahman1/ENGG_680_Project/blob/main/The_file_which_result_is_somehow_the_same_as_random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#here the new one

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Step 1: Load and Preprocess the Data
data = pd.read_excel('combined_data_final_EB (1) (1).xlsx')

# Convert 'date' and 'time' columns to datetime and extract features
data['date'] = pd.to_datetime(data['date'])
data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S')
data['hour'] = data['time'].dt.hour
data['minute'] = data['time'].dt.minute

# Encode categorical columns
data['day type'] = data['Day_Type'].map({'Sunny': 0, 'Rainy': 1, 'Snowy': 2})
data['weekend/holiday (y/n)'] = data['weekend/holiday (y/n)'].map({'y': 1, 'n': 0})
data['stampede (y/n)'] = data['stampede (y/n)'].map({'y': 1, 'n': 0})

# Convert 'date' to numerical features
data['day_of_year'] = data['date'].dt.dayofyear
data['time_minutes'] = data['hour'] * 60 + data['minute']

# Define features and target variable
features = ['day_of_year', 'time_minutes', 'weekend/holiday (y/n)', 'stampede (y/n)',
            'temp. (C)', 'daily precip. (mm)', 'wind (km/h)', 'day type']
X = data[features]
y = data['vehicle count']

# Scale features and target separately
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(X)
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))

# Prepare data for LSTM (creating sequences of past observations)
sequence_length = 24  # e.g., 24-hour sequence
X_lstm, y_lstm = [], []
for i in range(sequence_length, len(X_scaled)):
    X_lstm.append(X_scaled[i - sequence_length:i])
    y_lstm.append(y_scaled[i])

X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

# Step 2: Split Data into Training and Test Sets
X_train, X_test, y_train, y_test = train_test_split(X_lstm, y_lstm, test_size=0.2, random_state=35)

# Step 3: Build the LSTM Model
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Step 4: Evaluate the Model on the Test Set
y_pred = model.predict(X_test)

# Inverse transform predictions and actual values to original scale
y_test_rescaled = scaler_y.inverse_transform(y_test)
y_pred_rescaled = scaler_y.inverse_transform(y_pred)

# Calculate metrics
test_mse = mean_squared_error(y_test_rescaled, y_pred_rescaled)
test_mae = mean_absolute_error(y_test_rescaled, y_pred_rescaled)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test_rescaled, y_pred_rescaled)

print("LSTM Model Performance on Test Data:")
print(f"Mean Squared Error (MSE): {test_mse}")
print(f"Mean Absolute Error (MAE): {test_mae}")
print(f"Root Mean Squared Error (RMSE): {test_rmse}")
print(f"R-squared (R²): {test_r2}")

# Step 5: Function for User Input Prediction
def predict_vehicle_count():
    """
    Takes user input for features and returns predicted vehicle count.
    """
    # Request user input for each feature
    temp = float(input("Enter Temperature (C): "))
    precip = float(input("Enter Daily Precipitation (mm): "))
    wind = float(input("Enter Wind Speed (km/h): "))
    weekend = input("Is it a weekend or holiday? (y/n): ").strip().lower()
    weekend = 1 if weekend == 'y' else 0
    stampede = input("Is it during Stampede? (y/n): ").strip().lower()
    stampede = 1 if stampede == 'y' else 0
    day_type = input("Enter Day Type (Sunny/Rainy/Snowy): ").strip().capitalize()
    day_type = {'Sunny': 0, 'Rainy': 1, 'Snowy': 2}.get(day_type, 0)

    # Extract day and time from user-provided date (assuming YYYY-MM-DD format)
    date_str, time_str = input("Enter Date and Time (YYYY-MM-DD HH:MM:SS): ").split()
    date = pd.to_datetime(date_str)
    time = pd.to_datetime(time_str, format='%H:%M:%S')

    # Calculate day_of_year and time_minutes
    day_of_year = date.dayofyear
    time_minutes = time.hour * 60 + time.minute

    # Prepare input data in the correct format
    input_data = pd.DataFrame([[day_of_year, time_minutes, weekend, stampede, temp, precip, wind, day_type]], columns=features)
    input_scaled = scaler_X.transform(input_data)

    # Create the sequence expected by the LSTM model
    input_sequence = np.array([input_scaled[-sequence_length:]])  # Ensure it has the right shape

    # Make prediction
    predicted_scaled = model.predict(input_sequence)
    predicted_count = scaler_y.inverse_transform(predicted_scaled)[0][0]
    print(f"Predicted Vehicle Count: {predicted_count}")

# Run the function to get a prediction
predict_vehicle_count()


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


703/703 ━━━━━━━━━━━━━━━━━━━━ 27s 33ms/step - loss: 0.0171 - val_loss: 0.0061
Epoch 2/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 40s 32ms/step - loss: 0.0065 - val_loss: 0.0044
Epoch 3/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 42s 33ms/step - loss: 0.0045 - val_loss: 0.0030
Epoch 4/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 41s 33ms/step - loss: 0.0036 - val_loss: 0.0028
Epoch 5/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 41s 33ms/step - loss: 0.0037 - val_loss: 0.0025
Epoch 6/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 41s 34ms/step - loss: 0.0031 - val_loss: 0.0027
Epoch 7/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 39s 31ms/step - loss: 0.0034 - val_loss: 0.0028
Epoch 8/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 42s 32ms/step - loss: 0.0032 - val_loss: 0.0024
Epoch 9/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 23s 32ms/step - loss: 0.0032 - val_loss: 0.0029
Epoch 10/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 41s 32ms/step - loss: 0.0030 - val_loss: 0.0035
Epoch 11/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 42s 34ms/step - loss: 0.0032 - val_loss: 0.0027
Epoch 12/50
703/703 ━━━━━━━━━━━━━━━━━━━━ 